In [99]:
import pandas as pd
from pymongo import MongoClient
from apyori import apriori
import os

In [103]:
client = MongoClient(os.environ['MONGO_URL'])
db = client["adtProject"]
df1 = pd.DataFrame(db["resturant"].find({}))
df2 = pd.DataFrame(db["resturant"].find({}))

In [86]:
df1 = df1.rename(columns={'Order Number': 'Order ID'})


In [87]:
df1.shape

(74818, 6)

In [89]:
df2.shape

(119183, 6)

In [90]:
df = pd.concat([df1, df2])

In [98]:
dim = df.shape
dim

(13397, 60)

In [92]:
df.dtypes

Order id          float64
Order Date         object
Item Name          object
Quantity            int64
Product Price     float64
Total products      int64
Order Number      float64
dtype: object

In [93]:
df['Item Name'].nunique()

381

In [94]:
transformedDF = (df.groupby('Order id').head(len(df))
   .groupby('Order id')['Item Name'].agg(list)
).reset_index()
transformedDF

,Order id,Item Name
0,630.0,[Onion Bhaji]
1,647.0,[Onion Bhaji]
2,648.0,[Onion Bhaji]
3,651.0,[Onion Bhaji]
4,764.0,[Onion Bhaji]
...,...,...
13392,16114.0,"[Chicken Balti, Special Fried Rice]"
13393,16115.0,"[Chicken Pakora, Plain Papadum, Mango Chutney,..."
13394,16116.0,"[Aloo Chaat, Chicken Pakora, Chicken Chilli Ga..."
13395,16117.0,"[Plain Naan, Mushroom Rice, Tandoori Chicken (..."


In [95]:
df = pd.DataFrame([x for x in transformedDF['Item Name']])
df.columns = [x for x in df.columns]

In [96]:
df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,Onion Bhaji,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Onion Bhaji,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Onion Bhaji,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Onion Bhaji,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Onion Bhaji,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13392,Chicken Balti,Special Fried Rice,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13393,Chicken Pakora,Plain Papadum,Mango Chutney,Red Sauce,Korma - Prawn,Peshwari Naan,Pilau Rice,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13394,Aloo Chaat,Chicken Pakora,Chicken Chilli Garlic,Lamb Biryani,Keema Naan,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13395,Plain Naan,Mushroom Rice,Tandoori Chicken (1/4),Vindaloo - Lamb,Chapati,Lamb Tikka,Saag Paneer,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [100]:
rows = dim[0]
cols = dim[1]
transactions = []
for i in range(0, rows):
    transactions.append([str(df.values[i,j]) for j in range(0, cols)])

In [101]:
rule_list = apriori(transactions, min_support = 0.003, min_confidence = 0.1, min_lift = 3, min_length = 2)
results = list(rule_list)


In [102]:
bought_item = [tuple(result[2][0][0])[0] for result in results]
will_buy_item = [tuple(result[2][0][1])[0] for result in results]
support_values = [result[1] for result in results]
confidences = [result[2][0][2] for result in results]
lift_values = [result[2][0][3] for result in results]

In [ ]:
db['rules'].delete_many({})
for i in range(len(bought_item)):
    db['rules'].insert_one({
        "_id": i,
        "Bought Item":bought_item[i],
        "Expected To Be Bought":will_buy_item[i],
        "Support":support_values[i],
        "Confidence":confidences[i],
        "Lift":lift_values[i],
     })